# Nifty Behavior
When nifty opens too high or too low, does it maintain the direction throughout the day and for how long?
- How often nifty moves up and how often it moves down?
- In market shock, does market behave in a certain way?

In [8]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_START = dt.datetime.strptime("2015-02-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN60
INTERVAL = ut.INTERVAL_DAY
EXCHANGE = ut.EXCHANGE_NSE
pickle_file_name = "ocdf_2024_02_17.pkl"
# pickle_file_name = "test_analyzer_ocdf_2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
train_dates = all_dates
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

def get_symbol_first_candle_open(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open

def get_symbol_last_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[-1].close

train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["pd_close"] = train_dates.apply(lambda row: get_symbol_last_candle_close(SYMBOL, row.previous_trading_day), axis=1)
train_dates["td_open"] = train_dates.apply(lambda row: get_symbol_first_candle_open(SYMBOL, row.name), axis=1)
train_dates["td_close"] = train_dates.apply(lambda row: get_symbol_last_candle_close(SYMBOL, row.name), axis=1)
train_dates["market_move"] = train_dates.td_close - train_dates.td_open
train_dates["gap"] = train_dates.td_open - train_dates.pd_close

In [6]:
print(f"Up: {train_dates.loc[train_dates.gap >= 0].shape[0]}/{train_dates.shape[0]}")
print(f"Up PC: {train_dates.loc[train_dates.gap >= 0].shape[0] * 100/ train_dates.shape[0]}")
print(f"Avg UP: {train_dates.loc[train_dates.gap >= 0].gap.mean()}")
print(f"Down: {train_dates.loc[train_dates.gap < 0].shape[0]}/{train_dates.shape[0]}")
print(f"Down PC: {train_dates.loc[train_dates.gap < 0].shape[0] * 100/ train_dates.shape[0]}")
print(f"Avg DOWN: {train_dates.loc[train_dates.gap < 0].gap.mean()}")

Up: 497/741
Up PC: 67.0715249662618
Avg UP: 67.88655935613687
Down: 244/741
Down PC: 32.928475033738195
Avg DOWN: -79.39872950819681


In [8]:
train_dates

,previous_trading_day,pd_close,td_open,td_close,market_move,gap
trade_date,,,,,,
2015-02-02,2015-01-30,8808.90,8802.50,8797.40,-5.10,-6.40
2015-02-03,2015-02-02,8797.40,8823.15,8756.50,-66.65,25.75
2015-02-04,2015-02-03,8756.50,8789.15,8723.70,-65.45,32.65
2015-02-05,2015-02-04,8723.70,8733.10,8711.70,-21.40,9.40
2015-02-06,2015-02-05,8711.70,8696.85,8661.00,-35.85,-14.85
...,...,...,...,...,...,...
2023-02-20,2023-02-17,17944.20,17965.55,17844.60,-120.95,21.35
2023-02-21,2023-02-20,17844.60,17905.80,17826.70,-79.10,61.20
2023-02-22,2023-02-21,17826.70,17755.35,17554.30,-201.05,-71.35


In [12]:
print(train_dates[train_dates.market_move > 0].shape[0] / train_dates.shape[0])
print(train_dates[train_dates.market_move < 0].shape[0] / train_dates.shape[0])
print(train_dates.market_move.sum())
print(train_dates.market_move.mean())

0.4993252361673414
0.5006747638326585
-6716.880000000001
-9.064615384615387


In [13]:
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, gapup: {round(mdf[mdf.gap > 0].shape[0] / mdf.shape[0], 2)}, move_down: {round(mdf[mdf.gap < 0].shape[0] / mdf.shape[0], 2)}")
    print(f"Month: {month}, total gap: {round(mdf.gap.sum())}")
    print(f"Month: {month}, moveup: {round(mdf[mdf.market_move > 0].shape[0] / mdf.shape[0], 2)}, move_down: {round(mdf[mdf.market_move < 0].shape[0] / mdf.shape[0], 2)}")
    print(f"Month: {month}, total move: {round(mdf.market_move.sum())}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, gapup: {round(mdf[mdf.gap > 0].shape[0] / mdf.shape[0], 2)}, move_down: {round(mdf[mdf.gap < 0].shape[0] / mdf.shape[0], 2)}")
    print(f"Year: {year}, total gap: {round(mdf.gap.sum())}")
    print(f"Year: {year}, moveup: {round(mdf[mdf.market_move > 0].shape[0] / mdf.shape[0], 2)}, move_down: {round(mdf[mdf.market_move < 0].shape[0] / mdf.shape[0], 2)}")
    print(f"Year: {year}, total move: {round(mdf.market_move.sum())}")


Month: 2021-01-31 00:00:00, gapup: 0.65, move_down: 0.35
Month: 2021-01-31 00:00:00, total gap: 772
Month: 2021-01-31 00:00:00, moveup: 0.5, move_down: 0.5
Month: 2021-01-31 00:00:00, total move: -1119
Month: 2021-02-28 00:00:00, gapup: 0.75, move_down: 0.25
Month: 2021-02-28 00:00:00, total gap: 815
Month: 2021-02-28 00:00:00, moveup: 0.45, move_down: 0.55
Month: 2021-02-28 00:00:00, total move: 80
Month: 2021-03-31 00:00:00, gapup: 0.71, move_down: 0.29
Month: 2021-03-31 00:00:00, total gap: 890
Month: 2021-03-31 00:00:00, moveup: 0.48, move_down: 0.52
Month: 2021-03-31 00:00:00, total move: -728
Month: 2021-04-30 00:00:00, gapup: 0.68, move_down: 0.32
Month: 2021-04-30 00:00:00, total gap: 16
Month: 2021-04-30 00:00:00, moveup: 0.58, move_down: 0.42
Month: 2021-04-30 00:00:00, total move: -75
Month: 2021-05-31 00:00:00, gapup: 0.8, move_down: 0.2
Month: 2021-05-31 00:00:00, total gap: 685
Month: 2021-05-31 00:00:00, moveup: 0.65, move_down: 0.35
Month: 2021-05-31 00:00:00, total mov

In [3]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

train_dates["rolling_move"] = train_dates.market_move
train_dates["rolling_move_gap"] = train_dates.gap + train_dates.market_move
train_dates["rolling_gap"] = train_dates.gap

for i, row in train_dates.iterrows():
    # Update the value of the current row using the value of the previous row
    prev_row = train_dates.shift(1).loc[i]
    if not pd.isnull(prev_row.pd_close):
        train_dates.at[i, "rolling_move"] = prev_row.rolling_move + row["market_move"]
        train_dates.at[i, "rolling_move_gap"] = prev_row.rolling_move_gap + row["market_move"] + row.gap
        train_dates.at[i, "rolling_gap"] = prev_row.rolling_gap + row.gap

# train_dates["rolling_move"] = train_dates.rolling_move.rolling(window=2).apply(set_move)

In [4]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import CrosshairTool, Range1d

output_notebook()
TOOLS = "pan,crosshair,wheel_zoom,hover,box_zoom,reset,save"

p = figure(title="Bokeh Line Plot", x_axis_label='Index', y_axis_label='Value', x_axis_type="datetime", min_width=1500)
p.xaxis.ticker.desired_num_ticks = 40  # Tick every 5 minutes
crosshair_tool = CrosshairTool(
            dimensions="both",
            line_color="red",
            line_alpha=0.8,
        )
p.add_tools(crosshair_tool)

train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
mrgm = []
for month, mdf in gdf:
    cur = {
        "month": month,
        "gchange": mdf.iloc[-1].rolling_gap - mdf.iloc[0].rolling_gap,
        "mchange": mdf.iloc[-1].rolling_move - mdf.iloc[0].rolling_move,
        "rg1": mdf.iloc[0].rolling_gap,
        "rg2": mdf.iloc[-1].rolling_gap,
        "mg1": mdf.iloc[0].rolling_move,
        "mg2": mdf.iloc[-1].rolling_move,
    }
    mrgm.append(cur)

mrgm = pd.DataFrame(mrgm)
mrgm.set_index("month", inplace=True)

p.line(x=mrgm.index, y=mrgm.gchange, line_width=2)
show(p)

Loading BokehJS ...

In [17]:
print(mrgm.gchange.mean())
print(mrgm.mchange.mean())

276.5631958762906
-210.525463917528


In [14]:
mrgm

,gchange,mchange,rg1,rg2,mg1,mg2
month,,,,,,
2015-12-31,1951.67,-2859.91,-6.40,1945.27,-5.10,-2865.01
2016-12-31,1934.38,-1699.48,1937.37,3871.75,-2840.26,-4539.74
2017-12-31,3839.42,-1488.22,3896.05,7735.47,-4570.34,-6058.56
2018-12-31,2512.26,-2085.26,7736.47,10248.73,-6154.71,-8239.97
2019-12-31,4057.65,-2842.55,10267.88,14325.53,-8211.57,-11054.12
2020-12-31,5260.77,-3221.57,14359.23,19620.00,-11073.77,-14295.34
2021-12-31,7306.86,-3971.31,19634.35,26941.21,-14272.94,-18244.25
2022-12-31,1228.17,-748.57,26974.31,28202.48,-18005.70,-18754.27
2023-12-31,737.45,-1469.10,28228.88,28966.33,-18688.52,-20157.62


In [15]:
train_dates

,previous_trading_day,pd_close,td_open,td_close,market_move,gap,rolling_move,rolling_move_gap,rolling_gap,trade_date
trade_date,,,,,,,,,,
2015-02-02,2015-01-30,8808.90,8802.50,8797.40,-5.10,-6.40,-5.10,-11.50,-6.40,2015-02-02
2015-02-03,2015-02-02,8797.40,8823.15,8756.50,-66.65,25.75,-71.75,-52.40,19.35,2015-02-03
2015-02-04,2015-02-03,8756.50,8789.15,8723.70,-65.45,32.65,-137.20,-85.20,52.00,2015-02-04
2015-02-05,2015-02-04,8723.70,8733.10,8711.70,-21.40,9.40,-158.60,-97.20,61.40,2015-02-05
2015-02-06,2015-02-05,8711.70,8696.85,8661.00,-35.85,-14.85,-194.45,-147.90,46.55,2015-02-06
2015-02-09,2015-02-06,8661.00,8584.40,8526.29,-58.11,-76.60,-252.56,-282.61,-30.05,2015-02-09
2015-02-10,2015-02-09,8526.29,8478.10,8565.50,87.40,-48.19,-165.16,-243.40,-78.24,2015-02-10
2015-02-11,2015-02-10,8565.50,8603.29,8627.40,24.11,37.79,-141.05,-181.50,-40.45,2015-02-11
2015-02-12,2015-02-11,8627.40,8676.95,8711.50,34.55,49.55,-106.50,-97.40,9.10,2015-02-12


In [64]:
# train_dates.loc[train_dates.index.date >= dt.date(year=2021, month=1, day=1)]
train_dates[["rolling_move_gap", "rolling_gap", "gap", "market_move", "rolling_move", ]]
print(train_dates.shape)
print(train_dates.loc[train_dates.gap >= 25].shape[0] / train_dates.shape[0])
print(train_dates.loc[train_dates.gap < 25].shape)

(535, 10)
0.5065420560747663
(264, 10)


In [15]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)

train_dates.loc[train_dates.index.year == 2019]

,previous_trading_day,pd_close,td_open,td_close,market_move,gap
trade_date,,,,,,
2019-01-01,2018-12-31,10862.55,10881.70,10910.10,28.40,19.15
2019-01-02,2019-01-01,10910.10,10868.85,10792.50,-76.35,-41.25
2019-01-03,2019-01-02,10792.50,10796.80,10672.25,-124.55,4.30
2019-01-04,2019-01-03,10672.25,10699.70,10727.35,27.65,27.45
2019-01-07,2019-01-04,10727.35,10804.85,10771.80,-33.05,77.50
2019-01-08,2019-01-07,10771.80,10786.25,10802.15,15.90,14.45
2019-01-09,2019-01-08,10802.15,10862.40,10855.15,-7.25,60.25
2019-01-10,2019-01-09,10855.15,10859.35,10821.60,-37.75,4.20
2019-01-11,2019-01-10,10821.60,10834.75,10794.95,-39.80,13.15


In [12]:
ut.bokeh_plot(y=train_dates.market_move, x=train_dates.gap, y_label="market_move", x_label="gap")

Loading BokehJS ...

In [9]:
print(train_dates[train_dates.market_move < 0].shape[0])
print(train_dates[train_dates.market_move > 0].shape[0])

1175
1025


In [11]:
print(train_dates[train_dates.market_move < 0].market_move.mean())
print(train_dates[train_dates.market_move > 0].market_move.mean())

-76.25147234042564
66.91049756097551
